# 1. 라이브러리 임포트 및 환경 설정

In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


# 2. 데이터 및 액션 이름 로드

In [ ]:
dataset_path = os.path.join('create_dataset', 'dataset')

# seq_*.npy 파일 목록 가져오기
seq_files = [f for f in os.listdir(dataset_path) if f.startswith('seq_') and f.endswith('.npy')]

# 액션 이름 추출
actions = set()
for filename in seq_files:
    parts = filename.split('_')
    action_name = '_'.join(parts[1:-1])
    actions.add(action_name)

# 액션 이름을 리스트로 변환 및 정렬
actions = sorted(actions)
print(f"Detected actions: {actions}")

# 액션 이름 저장
np.save('actions.npy', np.array(actions))

# 액션 이름과 레이블 매핑 생성
action_to_label = {action: idx for idx, action in enumerate(actions)}
print(f"Action to label mapping: {action_to_label}")

# 3. 데이터 및 레이블 로드

In [ ]:
data_list = []
label_list = []

for filename in seq_files:
    parts = filename.split('_')
    action_name = '_'.join(parts[1:-1])
    label = action_to_label[action_name]

    seq_data = np.load(os.path.join(dataset_path, filename))
    data_list.append(seq_data)

    labels = np.full((seq_data.shape[0],), label)
    label_list.append(labels)

data = np.concatenate(data_list, axis=0)
labels = np.concatenate(label_list, axis=0)

print(f'Data shape: {data.shape}')
print(f'Labels shape: {labels.shape}')

# 4. 입력 데이터 및 레이블 준비

In [ ]:
x_data = data.astype(np.float32)
y_data = tf.keras.utils.to_categorical(labels, num_classes=len(actions))

print(f'x_data shape: {x_data.shape}')
print(f'y_data shape: {y_data.shape}')


# 5. 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_data, y_data, test_size=0.1, random_state=2021, stratify=labels
)

print(f'Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}')
print(f'Validation data shape: {x_val.shape}, Validation labels shape: {y_val.shape}')


# 6. 모델 구축

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 7. 모델 학습

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

os.makedirs('models', exist_ok=True)

callbacks = [
    ModelCheckpoint(
        'models/best_model.keras',
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=10,
        verbose=1,
        mode='max'
    )
]

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)

# 8. 학습 결과 시각화

In [ ]:
# 정확도
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# 손실
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

# 9. 모델 평가

In [ ]:
# 9. 모델 평가
best_model = tf.keras.models.load_model('models/best_model.keras')

val_loss, val_accuracy = best_model.evaluate(x_val, y_val, verbose=0)
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

# 10. 혼동 행렬 및 분류 보고서

In [ ]:
# 10. 혼동 행렬 및 분류 보고서
y_pred = best_model.predict(x_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

# 혼동 행렬
conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=actions, yticklabels=actions, cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# 분류 보고서
report = classification_report(y_true, y_pred_classes, target_names=actions)
print('Classification Report:\n', report)

# 11. 최종 모델 저장

In [ ]:
best_model.save('models/final_model.keras')